In [1]:
from tabulate import tabulate

data = [
    ['Falcon', '27-65', 'CX, U1, U2, U3 (now RZ, SX, X)', 'used in many public devices'],
    ['Hummingbird', '~65', 'CX, RZ, SX, X', 'Improved coherence and gate fidelity over Falcon'],
    ['Eagle', 127, 'CZ, RZ, SX, X','used in quantum utility benchmarks'],
    ['Heron', 133, 'CZ, RZ, SX, X', 'modular architecture'],
    ['Condor', 1121, 'CZ, RZ, SX, X', 'designed for scaling toward fault tolerance']
]

headers = ['Processor', 'Qubit count', 'Native gates', 'Features']

# Using the first row as headers
print(tabulate(data, headers=headers))

Processor    Qubit count    Native gates                    Features
-----------  -------------  ------------------------------  ------------------------------------------------
Falcon       27-65          CX, U1, U2, U3 (now RZ, SX, X)  used in many public devices
Hummingbird  ~65            CX, RZ, SX, X                   Improved coherence and gate fidelity over Falcon
Eagle        127            CZ, RZ, SX, X                   used in quantum utility benchmarks
Heron        133            CZ, RZ, SX, X                   modular architecture
Condor       1121           CZ, RZ, SX, X                   designed for scaling toward fault tolerance


Take the native gate sets —CZ, RZ, SX, X— into consideration this time.

In [2]:
# basics
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
# penny
import pennylane as qml
from pennylane.transforms import compile, decompose
from functools import partial
from pennylane.math import fidelity_statevector as fidelity_penny
# qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Gate
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_circuit_layout, circuit_drawer
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import state_fidelity as fidelity_qiskit
from math import pi
import copy

In [3]:
[ds] = qml.data.load("ketgpt")

In [24]:
random.seed(42)
# seed = random.randint(0,1000)
seed = 0

@partial(decompose, gate_set={qml.CZ, qml.CNOT, qml.RZ, qml.RY, qml.RX, qml.SX, qml.PauliX})
@qml.qnode(qml.device('default.qubit'))
def circuit(): # 0 to 999
    for op in ds.circuits[seed]:
        qml.apply(op)
    return qml.state()

In [25]:
def summary_penny(circuit):
    obj = qml.specs(circuit)()['resources']
    temp = qml.specs(circuit)()['resources'].gate_types # dict
    return [obj.num_wires, obj.num_gates, obj.gate_sizes[1], temp['CZ']+temp['CNOT'], temp['QubitUnitary'], obj.depth]
    
summary_penny(circuit)

[13, 1507, 1290, 217, 0, 437]

In [12]:
# compile
transpiled_circuit = compile(circuit)
summary_penny(transpiled_circuit)

[12, 1272, 1061, 211, 0, 398]

In [13]:
gate_name = []
gate_info = []
for op in ds.circuits[seed]:
    name = op.name
    gate_name.append(name)
    wires = [int(w) for w in op.wires]
    params = op.parameters
    gate_info.append({
        "name": name,
        "wires": wires,
        "params": params,
    })
    
gate_info[:3]

[{'name': 'Hadamard', 'wires': [0], 'params': []},
 {'name': 'Hadamard', 'wires': [1], 'params': []},
 {'name': 'Hadamard', 'wires': [2], 'params': []}]

In [14]:
def to_qiskit(qc, dict_elem):
    name = dict_elem['name']
    wire = dict_elem['wires']
    param = dict_elem['params']
    if name == 'Hadamard':
        qc.h(wire[0])
    if name == 'PauliX':
        qc.x(wire[0])
    if name == 'PauliY':
        qc.y(wire[0])
    if name == 'PauliZ':
        qc.z(wire[0])
    if name == 'CNOT':
        qc.cx(wire[0],wire[1])
    if name == 'CX':
        qc.cx(wire[0],wire[1])  
    if name == 'CY':
        qc.cy(wire[0],wire[1])  
    if name == 'CZ':
        qc.cz(wire[0],wire[1])    
    if name == 'QubitUnitary':
        qc.append(UnitaryGate(param[0]),wire)
    if name == 'U2':
        qc.u(pi/2, param[0], param[1], wire[0])

In [80]:
sample_q_num = qml.specs(circuit)()['resources'].num_wires
qc = QuantumCircuit(sample_q_num)
for dict_elem in gate_info:
    to_qiskit(qc, dict_elem)

In [81]:
# Use AerSimulator to extend the circuit
simulator = AerSimulator()
qc.save_statevector()  # Now this works!

# Run the simulation
result = simulator.run(qc).result()
statevector = result.data()['statevector']
ref_state = statevector.data

In [82]:
qc.num_qubits

13

In [83]:
def summary_qiskit(qc):
    counts = {'1q': 0, '2q': 0, 'U':0}
    for gate in qc.data:
        if len(gate.qubits) == 1:
            counts['1q'] += 1
        elif len(gate.qubits) == 2 and gate.name != 'unitary':
            counts['2q'] += 1
        elif len(gate.qubits) == 2 and gate.name == 'unitary':
            counts['U'] += 1
    return [qc.num_qubits, qc.size(), counts['1q'], counts['2q'], counts['U'], qc.depth()]

In [84]:
summary_qiskit(qc)

[13, 205, 116, 25, 64, 72]

In [113]:
# Transpile and simulate
def qiskit_transpiler(qc, level=2): # for lev. 2 and 3
    if level in [0, 1]:
        backend = Aer.get_backend('qasm_simulator')
        transpiled_qiskit = transpile(qc, backend, optimization_level=level)
        result = backend.run(transpiled_qiskit).result()
        statevector = result.get_statevector(qc)
        return  transpiled_qiskit, statevector
    elif level in [2, 3]:
        backend = Aer.get_backend('statevector_simulator')
        qc.data = [inst for inst in qc.data if inst.operation.name != 'save_statevector']
        basis_gates = ['cz', 'cx', 'rz', 'ry', 'rx', 'sx', 'x']
        transpiled_qiskit = transpile(qc, backend=backend, basis_gates=basis_gates, optimization_level=level)
        result = backend.run(transpiled_qiskit).result()
        statevector = result.get_statevector()
        return transpiled_qiskit, statevector
    else:
        print('check the optimization level')

# Run it
transpiled_qiskit, statevector = qiskit_transpiler(qc, level=0)
len(statevector)


C:\Users\syj43\AppData\Local\Temp\ipykernel_20404\826128710.py:22: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  len(statevector)


8192

In [86]:
def penny_from_qiskit(qc):
    pc_penny = qml.from_qiskit(qc)
    num_q = qc.num_qubits
    dev = qml.device("default.qubit", wires=num_q)
    @qml.qnode(dev)
    def circuit():
        pc_penny(wires=range(num_q))
        return qml.state()

    return circuit

penny_from_qiskit(qc)

<QNode: device='<default.qubit device (wires=13) at 0x24fe161d180>', interface='auto', diff_method='best'>

In [114]:
def make_table(qc): # takes qiskit circuits
    df = pd.DataFrame(columns=['original','qiskit_0','qiskit_1','qiskit_2','qiskit_3','penny'])
    
    df['original'] = summary_qiskit(qc)
    qc0 = copy.deepcopy(qc)
    qc1 = copy.deepcopy(qc)
    qc2 = copy.deepcopy(qc)
    qc3 = copy.deepcopy(qc)

    transpiled_qiskit_0, statevector_0 = qiskit_transpiler(qc0,0)
    df['qiskit_0'] = list(map(int,summary_qiskit(transpiled_qiskit_0)))
    transpiled_qiskit_1, statevector_1 = qiskit_transpiler(qc1,1)
    df['qiskit_1'] = list(map(int,summary_qiskit(transpiled_qiskit_1)))
    transpiled_qiskit_2, statevector_2 = qiskit_transpiler(qc2,2)
    df['qiskit_2'] = list(map(int,summary_qiskit(transpiled_qiskit_2)))
    transpiled_qiskit_3, statevector_3 = qiskit_transpiler(qc3,3)
    df['qiskit_3'] = list(map(int,summary_qiskit(transpiled_qiskit_3)))
    
    penny = penny_from_qiskit(qc)
    transpiled_penny  = compile(penny)
    df['penny'] = summary_penny(transpiled_penny)
    
    df.index = ['qubits', 'gates', '1q gates', '2q gates', 'unitary','depth']
    return transpiled_qiskit_0, statevector_0, transpiled_qiskit_1, statevector_1, transpiled_qiskit_2, statevector_2, transpiled_qiskit_3, statevector_3, transpiled_penny, df

In [115]:
transpiled_qiskit_0, statevector_0, transpiled_qiskit_1, statevector_1, transpiled_qiskit_2, statevector_2, transpiled_qiskit_3, statevector_3, transpiled_penny, df = make_table(qc)

c:\Users\syj43\AppData\Local\Programs\Python\Python310\lib\site-packages\pennylane_qiskit\converter.py:581: UserWarning: pennylane_qiskit.converter: The SaveStatevector instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(


In [116]:
df

,original,qiskit_0,qiskit_1,qiskit_2,qiskit_3,penny
qubits,13,13,13,13,13,12
gates,205,205,145,574,574,147
1q gates,116,116,62,445,445,64
2q gates,25,25,19,129,129,19
unitary,64,64,64,0,0,64
depth,72,72,59,201,201,60


In [118]:
def summary_qiskit_fidelity(transpiled_state, ref_state):
    return int(fidelity_qiskit(transpiled_state, ref_state))

In [124]:
summary_qiskit_fidelity(statevector_3, ref_state)

1

In [125]:
def summary_penny_fidelity(transpiled_circuit, circuit):
    num_q_original = qml.specs(transpiled_circuit)()['resources'].num_wires
    num_q_transpiled = qml.specs(circuit)()['resources'].num_wires
    print(num_q_original, num_q_transpiled)
    if num_q_original == num_q_transpiled:
        return fidelity_penny(transpiled_circuit, circuit)
    else:
        return None

In [128]:
# New row as a dictionary
new_row = {
        'original':'-', 
        'qiskit_0':summary_qiskit_fidelity(statevector_0, ref_state), 
        'qiskit_1':summary_qiskit_fidelity(statevector_1, ref_state), 
        'qiskit_2':summary_qiskit_fidelity(statevector_2, ref_state), 
        'qiskit_3':summary_qiskit_fidelity(statevector_3, ref_state), 
        'penny':''
           }
df = pd.concat([df, pd.DataFrame([new_row], index=['fidelity'])], ignore_index=False)
print(df)

         original  qiskit_0  qiskit_1  qiskit_2  qiskit_3 penny
qubits         13        13        13        13        13    12
gates         205       205       145       574       574   147
1q gates      116       116        62       445       445    64
2q gates       25        25        19       129       129    19
unitary        64        64        64         0         0    64
depth          72        72        59       201       201    60
fidelity        -         1         1         1         1      
